# Data Migration from CSV to PostgreSQL

In this notebook, we will perform a data migration from a CSV file to a PostgreSQL database. This process involves several key steps:

## Steps to Follow

1. **Configure the Environment**
2. **Import the Necessary Modules**
3. **Connect to the Database**
4. **Create an SQLAlchemy Session**
5. **Create the Candidates Table in the Database**
6. **Transform and Load the Data**

### Prerequisites

Ensure that your environment variables for database credentials are set up in a `.env` file. Additionally, install the required packages by running:

```bash
pip install -r requirements.txt

---

We will load the environment variables from the .env file, which contains important configurations such as paths and credentials. Then, we will obtain the working directory from these variables and add it to the system path to ensure that the project's modules can be imported correctly.

In [3]:
import sys
import os

load_dotenv()
work_dir = os.getenv('WORK_DIR')

sys.path.append(work_dir)

print('Workdir: ', work_dir)

Workdir:  /home/carol/workshop-02


We import the necessary modules and classes for the rest of the notebook.

In [10]:
from dotenv import load_dotenv
from db.db_connection import build_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import inspect
from models.model import GrammyAward
from sqlalchemy.exc import SQLAlchemyError
from src.transform.transform_grammy import TransformGrammy
import pandas as pd

The build_engine function is called to configure and create a connection to the PostgreSQL database.

In [5]:
engine = build_engine()

Successfully connected to the database workshop2!


A SQLAlchemy session is created using the database engine established in the previous step. Why? This session is necessary for performing read and write operations on the database.

In [6]:
Session = sessionmaker(bind=engine)
session = Session()

We check if the Grammy Awards table already exists in the database. If it does, the table is dropped. Then, a new Grammy Awards table is created. This ensures that the table is up-to-date and ready to receive new data. If any errors occur during this process, an error message is printed.

In [7]:
try:
    inspector = inspect(engine)

    if inspector.has_table('grammy_awards'):
        try:
            GrammyAward.__table__.drop(engine)
        except SQLAlchemyError as e:
            print(f"Error dropping table: {e}")
            raise
    try:
        GrammyAward.__table__.create(engine)
        print("Table creation was successful.")
    except SQLAlchemyError as e:
        print(f"Error creating table: {e}")
        raise

except SQLAlchemyError as error:
    print(f"An error occurred: {error}")

Table creation was successful.


An instance of the Transform class is created to read the file. Then, data transformations are applied, such as adding IDs. The transformed data is inserted into the Grammy awards table in the database. Finally, the database session is closed to free up resources.

In [13]:
grammy_awards = pd.read_csv('../data/the_grammy_awards.csv', sep=',' , encoding='utf-8')

try:
    df = TransformGrammy(grammy_awards)
    
    df.insert_ids()
    
    df.df.to_sql('grammy_awards', engine, if_exists='append', index=False)
    
    print("Data uploaded successfully")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if session:
        session.close()

Data uploaded successfully
